In [ ]:
#Import Dependencies
import pandas as pd
import os
import requests
import json
import pprint
import matplotlib.pyplot as plt
import numpy as np
from config import (aqs_api_key)
from config import (email_address)
# https://aqs.epa.gov/aqsweb/documents/data_api.html#signup for api information
pp = pprint.PrettyPrinter(indent = 4)
states_url = "https://aqs.epa.gov/data/api/list/states"
parameters_url = "https://aqs.epa.gov/data/api/list/classes"
sample_data_url = "https://aqs.epa.gov/data/api/dailyData/byState"
p1 = {"email" : email_address, "key" : aqs_api_key}
from sqlalchemy import create_engine

# States to be used and their ID Codes, All in top 10 for worst congestion and traffic
California = "06"
cali_data = []


In [ ]:
# Create a DataFrame for California and pull carbon monoxide data from API
cali_params = {"email" : email_address, "key" : aqs_api_key, "param" : "88101", 
          "bdate" : "20200901", "edate" : "20200923", "state" : California}

# Call to url
cali_response = requests.get(sample_data_url, params = cali_params).json()
cali_data.append(cali_response)
cali_response

# Round all data in cali_df to 3 demical places
cali_df = pd.DataFrame(cali_data[0]['Data']).round(3)
cali_df.head()

In [ ]:
# Clean the datafram by dropping unecessary collumns
clean_cali_df=cali_df.drop(["state_code","arithmetic_mean","cbsa", "pollutant_standard", "county_code","site_number",
                            "parameter_code","datum","cbsa_code","poc","method_code","local_site_name","event_type",
                            "validity_indicator","method_code"], axis=1)
clean_cali_df.head()

In [ ]:
# Create new column called "location" containing the latitude and logitude numbers.
clean_cali_df['location'] = clean_cali_df[['latitude', 'longitude']].apply(tuple, axis=1)
clean_cali_df

In [ ]:
#list(clean_cali_df.columns)

In [ ]:
# read in the CSV file for the fire locations data
csv_file = "../BootcampProject2/Resources/fire_locations.csv"
fire_data_df = pd.read_csv(csv_file)
fire_data_df.head()

In [ ]:
# Rename columns and round data to 3 decimal places
columns = ["state", "latitude", "longitude", "date_time", "pm25", "pm10", "co", 
                    "co2", "ch4", "nox", "nh3", "so2", "voc"]
CA_df = pd.DataFrame(fire_data_df).round(3)
CA_df = CA_df.loc[CA_df["state"] == "CA", columns]

# Create new Column that concatinates both the latitude and longitudinal data
CA_df['location'] = CA_df[['latitude', 'longitude']].apply(tuple, axis=1)
CA_df

In [ ]:
# Create Postgres Engine
engine = create_engine ('postgres://postgres:@localhost:5432/Bootcamp-Project-2')
# use own password and localhost id

In [ ]:
# Push both dataframes to Postgres Engine

In [ ]:
CA_df.to_sql ('fire_data', con = engine, index_label = 'fire_id')

In [ ]:
clean_cali_df.to_sql ('air_quality', con =engine)